# Cartpole DQN

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers


Using TensorFlow backend.


### Set Parameters

In [3]:
env = gym.make('CartPole-v0')

In [4]:
state_size = env.observation_space.shape[0]
state_size

4

In [5]:
action_size = env.action_space.n
action_size

2

In [6]:
batch_size = 32

In [28]:
n_episodes = 4000

In [8]:
output_dir = 'model_output/cartpole'

In [9]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Define Agent

In [29]:
class DQNAgent:
    
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        self.memory = deque(maxlen=2000)
        
        self.gamma = 0.95
        
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.001
        
        self.learning_rate = 0.001
        
        self.model = self.build_model()
        
    def build_model(self):
        model = Sequential();
        
        model.add(Dense(24, input_dim = self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        
        model.compile(loss='mse', optimizer=optimizers.Adam(lr=self.learning_rate))
        
        return model
    
    def remmember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            
            self.model.fit(state, target_f, epochs=1, verbose=0)
        
        if self.epsilon > self.epsilon_min:
            self.epsilon = self.epsilon * self.epsilon_decay
    
    def load(self, name):
        self.model.load_weights(name)
    
    def save(self, name):
        self.model.save_weights(name)

In [30]:
agent = DQNAgent(state_size, action_size)

### Interact with Environment

In [31]:
done = False

for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    
    for time in range(500):
        #env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        
        reward = reward if not done else -10
        
        next_state = np.reshape(next_state, [1, state_size])
        
        agent.remmember(state, action, reward, next_state, done)
        
        state = next_state
        
        if done:
            print("episodes: {}/{}, score: {}, e: {:.2}".format(e, n_episodes, time, agent.epsilon))
            break
            
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)
        
    if e % 50 == 0:
        agent.save(output_dir + " weights_" + '{:04d}'.format(e) + ".hdf5")
            

episodes: 0/4000, score: 14, e: 1.0
episodes: 1/4000, score: 29, e: 1.0
episodes: 2/4000, score: 18, e: 0.99
episodes: 3/4000, score: 37, e: 0.99
episodes: 4/4000, score: 14, e: 0.99
episodes: 5/4000, score: 33, e: 0.98
episodes: 6/4000, score: 10, e: 0.98
episodes: 7/4000, score: 10, e: 0.97
episodes: 8/4000, score: 13, e: 0.97
episodes: 9/4000, score: 13, e: 0.96
episodes: 10/4000, score: 14, e: 0.96
episodes: 11/4000, score: 25, e: 0.95
episodes: 12/4000, score: 15, e: 0.95
episodes: 13/4000, score: 29, e: 0.94
episodes: 14/4000, score: 12, e: 0.94
episodes: 15/4000, score: 11, e: 0.93
episodes: 16/4000, score: 17, e: 0.93
episodes: 17/4000, score: 29, e: 0.92
episodes: 18/4000, score: 10, e: 0.92
episodes: 19/4000, score: 41, e: 0.91
episodes: 20/4000, score: 19, e: 0.91
episodes: 21/4000, score: 9, e: 0.9
episodes: 22/4000, score: 14, e: 0.9
episodes: 23/4000, score: 21, e: 0.9
episodes: 24/4000, score: 20, e: 0.89
episodes: 25/4000, score: 10, e: 0.89
episodes: 26/4000, score: 33

episodes: 214/4000, score: 129, e: 0.34
episodes: 215/4000, score: 53, e: 0.34
episodes: 216/4000, score: 199, e: 0.34
episodes: 217/4000, score: 98, e: 0.34
episodes: 218/4000, score: 142, e: 0.34
episodes: 219/4000, score: 134, e: 0.34
episodes: 220/4000, score: 84, e: 0.33
episodes: 221/4000, score: 35, e: 0.33
episodes: 222/4000, score: 77, e: 0.33
episodes: 223/4000, score: 35, e: 0.33
episodes: 224/4000, score: 32, e: 0.33
episodes: 225/4000, score: 199, e: 0.33
episodes: 226/4000, score: 199, e: 0.32
episodes: 227/4000, score: 121, e: 0.32
episodes: 228/4000, score: 120, e: 0.32
episodes: 229/4000, score: 83, e: 0.32
episodes: 230/4000, score: 112, e: 0.32
episodes: 231/4000, score: 115, e: 0.32
episodes: 232/4000, score: 27, e: 0.31
episodes: 233/4000, score: 21, e: 0.31
episodes: 234/4000, score: 20, e: 0.31
episodes: 235/4000, score: 30, e: 0.31
episodes: 236/4000, score: 94, e: 0.31
episodes: 237/4000, score: 93, e: 0.31
episodes: 238/4000, score: 98, e: 0.3
episodes: 239/40

episodes: 422/4000, score: 178, e: 0.12
episodes: 423/4000, score: 129, e: 0.12
episodes: 424/4000, score: 126, e: 0.12
episodes: 425/4000, score: 129, e: 0.12
episodes: 426/4000, score: 169, e: 0.12
episodes: 427/4000, score: 199, e: 0.12
episodes: 428/4000, score: 156, e: 0.12
episodes: 429/4000, score: 150, e: 0.12
episodes: 430/4000, score: 132, e: 0.12
episodes: 431/4000, score: 168, e: 0.12
episodes: 432/4000, score: 133, e: 0.12
episodes: 433/4000, score: 141, e: 0.11
episodes: 434/4000, score: 131, e: 0.11
episodes: 435/4000, score: 119, e: 0.11
episodes: 436/4000, score: 144, e: 0.11
episodes: 437/4000, score: 128, e: 0.11
episodes: 438/4000, score: 127, e: 0.11
episodes: 439/4000, score: 124, e: 0.11
episodes: 440/4000, score: 126, e: 0.11
episodes: 441/4000, score: 131, e: 0.11
episodes: 442/4000, score: 143, e: 0.11
episodes: 443/4000, score: 106, e: 0.11
episodes: 444/4000, score: 161, e: 0.11
episodes: 445/4000, score: 174, e: 0.11
episodes: 446/4000, score: 139, e: 0.11


episodes: 625/4000, score: 198, e: 0.044
episodes: 626/4000, score: 147, e: 0.044
episodes: 627/4000, score: 144, e: 0.043
episodes: 628/4000, score: 139, e: 0.043
episodes: 629/4000, score: 168, e: 0.043
episodes: 630/4000, score: 133, e: 0.043
episodes: 631/4000, score: 133, e: 0.043
episodes: 632/4000, score: 149, e: 0.042
episodes: 633/4000, score: 143, e: 0.042
episodes: 634/4000, score: 157, e: 0.042
episodes: 635/4000, score: 145, e: 0.042
episodes: 636/4000, score: 121, e: 0.041
episodes: 637/4000, score: 164, e: 0.041
episodes: 638/4000, score: 115, e: 0.041
episodes: 639/4000, score: 108, e: 0.041
episodes: 640/4000, score: 118, e: 0.041
episodes: 641/4000, score: 91, e: 0.04
episodes: 642/4000, score: 95, e: 0.04
episodes: 643/4000, score: 92, e: 0.04
episodes: 644/4000, score: 87, e: 0.04
episodes: 645/4000, score: 117, e: 0.04
episodes: 646/4000, score: 89, e: 0.039
episodes: 647/4000, score: 63, e: 0.039
episodes: 648/4000, score: 75, e: 0.039
episodes: 649/4000, score: 8

episodes: 827/4000, score: 199, e: 0.016
episodes: 828/4000, score: 199, e: 0.016
episodes: 829/4000, score: 199, e: 0.016
episodes: 830/4000, score: 199, e: 0.016
episodes: 831/4000, score: 199, e: 0.016
episodes: 832/4000, score: 167, e: 0.016
episodes: 833/4000, score: 111, e: 0.015
episodes: 834/4000, score: 199, e: 0.015
episodes: 835/4000, score: 188, e: 0.015
episodes: 836/4000, score: 199, e: 0.015
episodes: 837/4000, score: 199, e: 0.015
episodes: 838/4000, score: 199, e: 0.015
episodes: 839/4000, score: 149, e: 0.015
episodes: 840/4000, score: 165, e: 0.015
episodes: 841/4000, score: 199, e: 0.015
episodes: 842/4000, score: 121, e: 0.015
episodes: 843/4000, score: 189, e: 0.015
episodes: 844/4000, score: 176, e: 0.015
episodes: 845/4000, score: 191, e: 0.015
episodes: 846/4000, score: 146, e: 0.014
episodes: 847/4000, score: 155, e: 0.014
episodes: 848/4000, score: 12, e: 0.014
episodes: 849/4000, score: 190, e: 0.014
episodes: 850/4000, score: 172, e: 0.014
episodes: 851/400

...

episodes: 3180/4000, score: 199, e: 0.001
episodes: 3181/4000, score: 199, e: 0.001
episodes: 3182/4000, score: 199, e: 0.001
episodes: 3183/4000, score: 199, e: 0.001
episodes: 3184/4000, score: 14, e: 0.001
episodes: 3185/4000, score: 141, e: 0.001
episodes: 3186/4000, score: 140, e: 0.001
episodes: 3187/4000, score: 188, e: 0.001
episodes: 3188/4000, score: 182, e: 0.001
episodes: 3189/4000, score: 149, e: 0.001
episodes: 3190/4000, score: 145, e: 0.001
episodes: 3191/4000, score: 149, e: 0.001
episodes: 3192/4000, score: 199, e: 0.001
episodes: 3193/4000, score: 183, e: 0.001
episodes: 3194/4000, score: 199, e: 0.001
episodes: 3195/4000, score: 199, e: 0.001
episodes: 3196/4000, score: 199, e: 0.001
episodes: 3197/4000, score: 151, e: 0.001
episodes: 3198/4000, score: 199, e: 0.001
episodes: 3199/4000, score: 199, e: 0.001
episodes: 3200/4000, score: 199, e: 0.001
episodes: 3201/4000, score: 199, e: 0.001
episodes: 3202/4000, score: 199, e: 0.001
episodes: 3203/4000, score: 199, e:

episodes: 3377/4000, score: 126, e: 0.001
episodes: 3378/4000, score: 148, e: 0.001
episodes: 3379/4000, score: 199, e: 0.001
episodes: 3380/4000, score: 199, e: 0.001
episodes: 3381/4000, score: 199, e: 0.001
episodes: 3382/4000, score: 178, e: 0.001
episodes: 3383/4000, score: 179, e: 0.001
episodes: 3384/4000, score: 180, e: 0.001
episodes: 3385/4000, score: 150, e: 0.001
episodes: 3386/4000, score: 162, e: 0.001
episodes: 3387/4000, score: 168, e: 0.001
episodes: 3388/4000, score: 163, e: 0.001
episodes: 3389/4000, score: 159, e: 0.001
episodes: 3390/4000, score: 129, e: 0.001
episodes: 3391/4000, score: 128, e: 0.001
episodes: 3392/4000, score: 113, e: 0.001
episodes: 3393/4000, score: 102, e: 0.001
episodes: 3394/4000, score: 100, e: 0.001
episodes: 3395/4000, score: 103, e: 0.001
episodes: 3396/4000, score: 103, e: 0.001
episodes: 3397/4000, score: 110, e: 0.001
episodes: 3398/4000, score: 102, e: 0.001
episodes: 3399/4000, score: 107, e: 0.001
episodes: 3400/4000, score: 118, e

episodes: 3573/4000, score: 199, e: 0.001
episodes: 3574/4000, score: 199, e: 0.001
episodes: 3575/4000, score: 199, e: 0.001
episodes: 3576/4000, score: 199, e: 0.001
episodes: 3577/4000, score: 199, e: 0.001
episodes: 3578/4000, score: 199, e: 0.001
episodes: 3579/4000, score: 199, e: 0.001
episodes: 3580/4000, score: 199, e: 0.001
episodes: 3581/4000, score: 199, e: 0.001
episodes: 3582/4000, score: 199, e: 0.001
episodes: 3583/4000, score: 199, e: 0.001
episodes: 3584/4000, score: 199, e: 0.001
episodes: 3585/4000, score: 199, e: 0.001
episodes: 3586/4000, score: 199, e: 0.001
episodes: 3587/4000, score: 12, e: 0.001
episodes: 3588/4000, score: 199, e: 0.001
episodes: 3589/4000, score: 199, e: 0.001
episodes: 3590/4000, score: 113, e: 0.001
episodes: 3591/4000, score: 199, e: 0.001
episodes: 3592/4000, score: 199, e: 0.001
episodes: 3593/4000, score: 199, e: 0.001
episodes: 3594/4000, score: 168, e: 0.001
episodes: 3595/4000, score: 153, e: 0.001
episodes: 3596/4000, score: 165, e:

episodes: 3770/4000, score: 199, e: 0.001
episodes: 3771/4000, score: 190, e: 0.001
episodes: 3772/4000, score: 199, e: 0.001
episodes: 3773/4000, score: 199, e: 0.001
episodes: 3774/4000, score: 181, e: 0.001
episodes: 3775/4000, score: 199, e: 0.001
episodes: 3776/4000, score: 199, e: 0.001
episodes: 3777/4000, score: 199, e: 0.001
episodes: 3778/4000, score: 199, e: 0.001
episodes: 3779/4000, score: 199, e: 0.001
episodes: 3780/4000, score: 105, e: 0.001
episodes: 3781/4000, score: 199, e: 0.001
episodes: 3782/4000, score: 199, e: 0.001
episodes: 3783/4000, score: 188, e: 0.001
episodes: 3784/4000, score: 199, e: 0.001
episodes: 3785/4000, score: 199, e: 0.001
episodes: 3786/4000, score: 186, e: 0.001
episodes: 3787/4000, score: 125, e: 0.001
episodes: 3788/4000, score: 137, e: 0.001
episodes: 3789/4000, score: 122, e: 0.001
episodes: 3790/4000, score: 121, e: 0.001
episodes: 3791/4000, score: 130, e: 0.001
episodes: 3792/4000, score: 159, e: 0.001
episodes: 3793/4000, score: 199, e

episodes: 3967/4000, score: 185, e: 0.001
episodes: 3968/4000, score: 199, e: 0.001
episodes: 3969/4000, score: 199, e: 0.001
episodes: 3970/4000, score: 199, e: 0.001
episodes: 3971/4000, score: 199, e: 0.001
episodes: 3972/4000, score: 180, e: 0.001
episodes: 3973/4000, score: 171, e: 0.001
episodes: 3974/4000, score: 195, e: 0.001
episodes: 3975/4000, score: 199, e: 0.001
episodes: 3976/4000, score: 199, e: 0.001
episodes: 3977/4000, score: 171, e: 0.001
episodes: 3978/4000, score: 150, e: 0.001
episodes: 3979/4000, score: 175, e: 0.001
episodes: 3980/4000, score: 174, e: 0.001
episodes: 3981/4000, score: 166, e: 0.001
episodes: 3982/4000, score: 151, e: 0.001
episodes: 3983/4000, score: 140, e: 0.001
episodes: 3984/4000, score: 139, e: 0.001
episodes: 3985/4000, score: 194, e: 0.001
episodes: 3986/4000, score: 183, e: 0.001
episodes: 3987/4000, score: 182, e: 0.001
episodes: 3988/4000, score: 172, e: 0.001
episodes: 3989/4000, score: 171, e: 0.001
episodes: 3990/4000, score: 188, e